<a href="https://colab.research.google.com/github/JankovitsA/FAKT_MI/blob/main/fakt_hf_4_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Részvény prediktálás

In [2]:
import pandas as pd
from google.colab import drive
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
drive.mount('/content/drive')
otp = pd.read_csv('/content/drive/My Drive/progfakt/otp.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
otp.head()

,Név,Dátum,Utolsó ár,Mennyiség
0,OTP,2020.02.12.,14660.0,356596.0
1,OTP,2020.02.13.,14990.0,343220.0
2,OTP,2020.02.14.,15150.0,546600.0
3,OTP,2020.02.17.,15250.0,119497.0
4,OTP,2020.02.18.,15250.0,275739.0


In [12]:
#Az utolsó árat fogom az előzőből (mennyiséggel együtt) prediktálni, ha minden igaz

In [15]:
otp2=otp.copy()
otp2=otp2['Utolsó ár']
otp2.head()

0    14660.0
1    14990.0
2    15150.0
3    15250.0
4    15250.0
Name: Utolsó ár, dtype: float64

In [16]:
def split_sequence(sequence, n_steps):
	x, y = list(), list()
	for i in range(len(sequence)):
		end_ix = i + n_steps # Az adott szekvencia-darab utolsó eleme
		if end_ix > len(sequence)-1: # Ellenőrizzük nem értünk-e a teljes lista végére
			break
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix] # Kimentjük a szekvencia darabot és az utolsó elemet mint eredményváltozó
		x.append(seq_x)
		y.append(seq_y)
	return np.array(x), np.array(y)

In [17]:
x, y = split_sequence(otp2, 10)

In [ ]:
#ellenőrzés
for i in range(len(x)): # Kiprintelem a jobb értelmezhetőségért
	print(x[i], y[i])

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)

In [20]:
#normalizálás
normalizer = StandardScaler()
normalizer.fit(x_train)
x_train = normalizer.transform(x_train)
x_test = normalizer.transform(x_test)

In [21]:
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, Flatten, Dense, Dropout, Conv1D, MaxPooling1D, AveragePooling1D
import keras
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [33]:
input = Input(shape=(10,1)) #jobb híján az Alexhez hasonlóan csináltam, még utána kell olvasnom...

x = Conv1D(filters=64, kernel_size=3, strides=1, activation='relu')(input)
x = MaxPooling1D(pool_size=2, strides=1)(x)
x = Conv1D(filters=256, kernel_size=2, activation='relu', strides=1)(x) #padding='same'
x = MaxPooling1D(pool_size=2, strides=1)(x)
x = Conv1D(filters=384, kernel_size=2, activation='relu', strides=1)(x) #padding='same'
x = Conv1D(filters=384, kernel_size=2, activation='relu', strides=1)(x)
x = Conv1D(filters=192, kernel_size=2, activation='relu', strides=1)(x)
x = MaxPooling1D(pool_size=2, strides=1)(x)



x = Flatten()(x)

x = Dense(4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.4)(x)

output = Dense(1, activation='softmax')(x)

In [34]:
model = keras.Model(input, output)
model.compile(optimizer='adam',
    loss='MSE')

In [35]:
AlexNet = keras.Model(input, output)
early_stopping_callback = EarlyStopping(patience=5, restore_best_weights=True)
tensorboard_callback = TensorBoard('logs/run1')

In [36]:
AlexNet.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
AlexNet.fit(x_train, y_train, epochs=100, validation_split=0.1,callbacks=[early_stopping_callback, tensorboard_callback]) #nem igazán látszik működni

Epoch 1/100
12/12 [==============================] - 8s 513ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/100
12/12 [==============================] - 4s 303ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/100
12/12 [==============================] - 4s 357ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/100
12/12 [==============================] - 4s 326ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/100
12/12 [==============================] - 5s 453ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00


In [ ]:
AlexNet.predict(x_test)
AlexNet.evaluate(x_test, y_test)